In [ ]:
# from langchain_ollama import ChatOllama

# mistral = ChatOllama(
#     model="mistral",
#     temperature=0,
# )

In [ ]:
# from langchain_ollama import OllamaEmbeddings

# embeddings = OllamaEmbeddings(
#     model="mistral",
# )

###  Using Groqcloud API

In [125]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("GROQ_API_KEY")

In [127]:
from langchain_groq import ChatGroq

groqLLM = ChatGroq(
    model="deepseek-r1-distill-qwen-32b", 
    api_key=api_key,
    temperature=0.7, 
    max_tokens=512    # Limit response length
)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import tqdm as notebook_tqdm

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [ ]:
result = embeddings.embed_query("hello")

In [ ]:
len(result)

In [128]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [129]:
import time

index_name = "personal" 

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
# index.delete(delete_all=True)

In [130]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader

def custom_loader(file_path: str):
    if file_path.endswith(".pdf"):
        return PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        return TextLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_path}")

loader = DirectoryLoader("personal", glob="**/*", show_progress=True, loader_cls=custom_loader)
docs = loader.load()

In [ ]:
# vector_store.add_documents(docs)

In [131]:
groq_retreiver = vector_store.as_retriever(search_kwargs={'k':2})

In [133]:
from langchain_core.tools import tool

@tool
def marks_reader(self, file_path:str) -> str:
    """
    Reads the content of a specified text file and returns it as a string.

    Args:
        file_path (str): The path to the text file to be read.

    Returns:
        str: The content of the file if successfully read, or an error message if the file is not found or an exception occurs.

    Raises:
        FileNotFoundError: If the file does not exist at the specified path.
        Exception: For other file-reading errors (e.g., permission issues, encoding errors), with details in the returned string.
    """
    try:
        if not os.path.exists(file_path):
            return f"Error: File not found at path {file_path}"
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        return content
    except Exception as e:
        return f"Error reading file: {str(e)}"
    
@tool
def write_to_file(file_path: str, content: str, append: bool = False) -> str:
    """
    Write or append content to a specified file.
    
    Args:
        file_path (str): The path to the file.
        content (str): The content to write into the file.
        append (bool): If True, appends to the file; if False, overwrites it.
    
    Returns:
        str: Confirmation message or error if it fails.
    """
    try:
        mode = "a" if append else "w"
        with open(file_path, mode, encoding="utf-8") as f:
            f.write(content)
        return f"Successfully {'appended to' if append else 'wrote to'} {file_path}"
    except Exception as e:
        return f"Error writing to file: {str(e)}"
    
    
tools = [marks_reader, write_to_file]
groqLLM_with_tools = groqLLM.bind_tools(tools)

In [138]:
from langchain.prompts import PromptTemplate

from langchain_core.prompts import ChatPromptTemplate

template = """
You are digitalME, an AI assistant created by Berojgar & company, designed to engage in natural, helpful conversations. You can answer general questions, provide insights, and assist with tasks. When relevant, you have access to a vector database of documents and tools to enhance your responses.

### Chat History:
{messages}

### User Query:
{user_query}

### Retrieved Documents (from vector database, if applicable):
{retrieved_documents}

### Instructions:
- Engage in a natural, conversational tone as a helpful assistant.
- Use the chat history to maintain context and provide coherent, relevant responses.
- If the user query can be answered directly based on general knowledge or chat history, do so concisely and clearly.
- If the query relates to information in the retrieved documents, incorporate that information into your response and briefly note that it came from the documents (e.g., "Based on your documents...").
- If the query requires additional information or functionality (e.g., reading a file or writing to a file), use the available tools, execute them fully, and include their results in your response. Do NOT output raw JSON tool calls; instead, summarize the tool's output (e.g., "I used the file_reader tool to check your sports from sports.txt, and here's what I found: [result]", or I have succesfully written [text] in the file [file_name]).
- If a tool returns an error (e.g., file not found), report it clearly in the response (e.g., "I tried using the file_reader tool, but the file sports.txt wasn’t found", or [file_name couldn't be created to write]).
- If no retrieved documents or tools are relevant, proceed with a general conversational response.
- Keep your answers friendly, concise, and tailored to the user's intent.

### Response:
Provide your conversational response here, blending general knowledge, document insights, and tool outputs as needed.
"""

prompt = ChatPromptTemplate.from_template(template)

In [139]:
chain = prompt | groqLLM_with_tools

In [ ]:
# user_query = input("Your query :: \n")
# retrieved_documents = mistral_retriever.invoke(user_query)

# chain = prompt | mistral_with_tools

# response = chain.invoke({"user_query": user_query, "retrieved_documents": retrieved_documents})
# print(response.content)

In [ ]:
# print(response.content)

### Let's add memory to the chatbot using **langgraph**

In [142]:
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import START, END, StateGraph
import operator
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

class GraphState(TypedDict):
    user_query : str
    retrieved_documents : Sequence[str]
    messages: Annotated[list, add_messages]


def input_node(state: GraphState) -> GraphState:
    if not state.get("messages"):
        state["messages"] = [HumanMessage(content=state["user_query"])]
    return state

def retrieval_node(state: GraphState)->GraphState:
    retrieved_documents = groq_retreiver.invoke(state['user_query'])
    state['retrieved_documents'] = [doc.page_content for doc in retrieved_documents]
    return state

def processing_node(state: GraphState)->GraphState:
    chain = prompt | groqLLM_with_tools
    response = chain.invoke(
        {
            "messages": state["messages"],
            "user_query": state['user_query'],
            "retrieved_documents": state['retrieved_documents']
        }
    )
    state["messages"] = state["messages"] + [response]
    return state

tool_node = ToolNode(tools=tools)

workflow = StateGraph(GraphState)

workflow.add_node("input", input_node)
workflow.add_node("retrieval", retrieval_node)
workflow.add_node("processing", processing_node)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "input")
workflow.add_edge("input","retrieval")
workflow.add_edge("retrieval","processing")

workflow.add_conditional_edges("processing", tools_condition)  # if tool call is there, it is called
workflow.add_edge("tools", "processing")  # if toolcall is made then tool execution should occur, if occur, this is ..

workflow.add_edge("processing",END)

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

In [144]:
config = {"configurable": {"thread_id": "1"}} 

print("Start chatting! Type 'exit' to quit.")
while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("Goodbye!")
        break
    
    initial_state = {
        "user_query": user_query,
        "retrieved_documents": [],
        "messages": app.get_state(config).values.get("messages", [])  # Load previous messages
    }
    
    result = app.invoke(initial_state, config=config)
    print("Assistant:", result["messages"][-1].content)

Start chatting! Type 'exit' to quit.
Assistant: 
Assistant: I'll help you read the contents of the `output.txt` file. I used the `marks_reader` tool to read the file, and here's the content:

```
xxxxxx
```

If you need any further assistance, feel free to ask!
Assistant: I used the `marks_reader` tool to read the content of the `output.txt` file. Here's the content:

```
xxxxxx
```

If you need any further assistance, feel free to ask!
Goodbye!
